# Convertible Bond Valuation - QL Example

Value a Convertible Bond and compare to QL example at http://gouthamanbalaraman.com/blog/value-convertible-bond-quantlib-python.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.bonds import *
from financepy.market.curves import *

####################################################################
# FINANCEPY BETA Version 0.260 - This build:  22 Nov 2022 at 12:35 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating the Convertible Bond

In [3]:
valuation_date = Date(9, 1, 2004)
settlement_date = valuation_date.add_days(2)

We set the maturity date, annualised coupon, coupon annual_frequency and convention for calculating accrued interest

In [4]:
maturity_date = Date(15, 3, 2022)
coupon = 0.0575
frequencyType = FrequencyTypes.SEMI_ANNUAL
accrualBasis = DayCountTypes.THIRTY_360_BOND
face = 100.0 
conversion_ratio = 3.84615  

We need to set the face amount and the conversion ratio. This is the number of shares you can convert to for that face amount.

We can also delay the start of the conversion feature using the start convert date. We allow conversion to occur from settlement.

In [5]:
start_convert_date = settlement_date

We can also set the call schedule - the price and date on which the issuer can call back the bond. This is given with reference to a price of 100. Making the call price very high means it will never be called. So the call has no impact.

In [6]:
call_price = 100.0
call_dates = [Date(20, 3, 2007), Date(15, 3, 2012), Date(15, 3, 2017)]
call_prices = [call_price, call_price, call_price]

We can also set the put schedule - the price and date on which the owner can put back the bond to the issuer. This is given with reference to a price of 100. Making the put price very low means it will never be put.

In [7]:
putPrice = 100.0
put_dates = [Date(20, 3, 2007), Date(15, 3, 2012), Date(15, 3, 2017)]
put_prices = [putPrice, putPrice, putPrice]

At this point we can create our bond.

In [8]:
bond = BondConvertible(maturity_date, coupon, frequencyType, start_convert_date, conversion_ratio,
                          call_dates, call_prices, put_dates, put_prices, accrualBasis, face)

In [9]:
print(bond)

OBJECT TYPE: BondConvertible
MATURITY DATE: 15-MAR-2022
COUPON: 0.0575
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: DayCountTypes.THIRTY_360_BOND
FACE AMOUNT: 100.0
CONVERSION RATIO: 3.84615
START CONVERT DATE: 11-JAN-2004
CALL: DATES
20-MAR-2007: 100.0
15-MAR-2012: 100.0
15-MAR-2017: 100.0
PUT: DATES
20-MAR-2007: 100.0
15-MAR-2012: 100.0
15-MAR-2017: 100.0



## Valuation Inputs

The model allows a discrete dividend schedule. We can use the FinSchedule class to do this.

In [10]:
dividendSchedule = Schedule(settlement_date, maturity_date).schedule_dates()[1:]
dividend_yields = [0.02] * len(dividendSchedule)

In [11]:
rate = 0.04
discount_curve = DiscountCurveFlat(settlement_date, rate, FrequencyTypes.CONTINUOUS)

In [12]:
stock_price = 29.04
stock_volatility = 0.40
credit_spread = 0.03
recovery_rate = 0.40 # 40 percent
num_steps_per_year = 1000

In [13]:
bond.value(valuation_date, stock_price, stock_volatility, dividendSchedule, dividend_yields,
           discount_curve, credit_spread, recovery_rate, num_steps_per_year)

{'cbprice': 135.3545239908085,
 'bond': 89.44004150041098,
 'delta': 2.8964034274730013,
 'gamma': 0.5948917917094878,
 'theta': 221.62044995424162}

Copyright (c) 2020 Dominic O'Kane

This compares to 132.31 found by QL. The difference could be due to the different treatment of dividends or the credit spread.